# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'classDB', 'config', 'gardenDB', 'local', 'mydatabase', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [6]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [7]:
# assign the collection to a variable
establishments = db['establishments']

In [8]:
# review a document in the establishments collection
pprint(establishments.find_one())



{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Tea Room',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 551803,
 'LocalAuthorityBusinessID': 'PI/000070948',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67590a6514e0db8715b9b189'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/551803',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate':

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
	"BusinessName": "Penang Flavours",
	"BusinessType": "Restaurant/Cafe/Canteen",
	"AddressLine1": "Penang Flavours",
	"AddressLine2": "Greenwich",
	"AddressLine3": "",
	"AddressLine4": "",
	"PostCode": "SE10 9HY",
	"Phone": "",
	"RatingValue": None,
	"LocalAuthorityName": "Greenwich"
}


In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)


InsertOneResult(ObjectId('6759c624f2d6ccd397e9b798'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': 'Greenwich',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'LocalAuthorityName': 'Greenwich',
 'Phone': '',
 'PostCode': 'SE10 9HY',
 'RatingValue': None,
 '_id': ObjectId('6759c624f2d6ccd397e9b798')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
	{"BusinessType": "Restaurant/Cafe/Canteen"},
	{"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(business_type)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
	{"BusinessName": "Penang Flavours"},
	{"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}}
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': 'Greenwich',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Greenwich',
 'Phone': '',
 'PostCode': 'SE10 9HY',
 'RatingValue': None,
 '_id': ObjectId('6759c624f2d6ccd397e9b798')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(dover_count)


994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})


DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [17]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(remaining_dover_count)


0


In [18]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67590a6514e0db8715b9b46e'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

# Update longitude
establishments.update_many(
	{"geocode.longitude": {"$type": "string"}},
	[{"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}}}]
)

# Update latitude
establishments.update_many(
	{"geocode.latitude": {"$type": "string"}},
	[{"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"}}}]
)


UpdateResult({'n': 38785, 'nModified': 38785, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
	{"RatingValue": {"$type": "string"}},
	[{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one({}, {"geocode.longitude": 1, "geocode.latitude": 1, "RatingValue": 1, "_id": 0})
pprint(sample_document)


{'RatingValue': 5,
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')}}
